In [4]:
import os
import numpy as np
from time import time
import datetime
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd

# Pleasure

In [5]:
df = pd.read_csv('Continuous_fusion_4/fusion_4_pleasure.csv')
df

,opensmile_prediction_p,hog_prediction_p,vggish_prediction_p,lstm_p_prediction,groundtruth_p
0,0.115646,0.122450,0.275033,0.159465,0.00
1,0.112203,-0.056877,0.292502,0.031091,0.00
2,0.123905,0.298645,-0.036120,0.346203,0.75
3,0.261675,-0.085651,0.035924,-0.068999,0.00
4,-0.051422,0.708457,0.042538,0.967095,1.00
...,...,...,...,...,...
4380,-0.036468,0.148861,-0.125886,-0.121085,0.00
4381,0.175664,0.363688,0.297178,0.258295,0.50
4382,0.041683,0.147541,-0.031547,-0.017166,0.00
4383,0.001698,-0.085040,0.133264,-0.061091,0.00


In [6]:
x = df.iloc[:, :-1]  # 数据特征
y = df.iloc[:,-1]  # 标签
# 将数据划分为训练集和测试集，test_size=.3表示30%的测试集, 随机数种子, 保证可复现性
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=420)

# 修正测试集和训练集的索引
for i in [x_train, x_test, y_train, y_test ]:
    i.index  = range(i.shape[0])

# 标准化
scaler_x = StandardScaler()
# scaler_y = StandardScaler()
x_train_std = scaler_x.fit_transform(x_train)
x_test_std = scaler_x.fit_transform(x_test)
# y_train = scaler_y.fit_transform(np.array(y_train).reshape([-1,1])).reshape(-1)
# y_test = scaler_y.fit_transform(np.array(y_test).reshape([-1,1])).reshape(-1)
print(x_train_std.shape)
print(y_train.shape)

(3069, 4)
(3069,)


In [7]:
def getPvar(vals, mean):
    N = len(vals)
    su = 0
    for i in range(len(vals)):
        su = su + ((vals[i]-mean)*(vals[i]-mean))
    pvar = (1/N) * su
    return pvar

def getMean(vals):
    su = 0
    for i in range(len(vals)):
        su = su + vals[i]
    mean = su/(len(vals))
    return mean

def getMeanofDiffs(xvals, yvals):
    su = 0
    for i in range(len(xvals)):
        su = su + ((xvals[i] - yvals[i])*(xvals[i] - yvals[i]))
    meanodiffs = su/(len(xvals))
    return meanodiffs

def getCCC(pvarfe,pvarexp,meanofdiff,meanfe,meanexp):
    bottom = pvarfe + pvarexp + ((meanfe - meanexp)*(meanfe - meanexp))
    answer = 1 - (meanofdiff / bottom)
    return answer

In [8]:
times_all = time()
# 调试两个参数 gamma & C  ，默认情况下C为1，通常来说这都是一个合理的参数。
RMSE_list = []
Spearman_list = []
CCC_list = []

gamma_range = np.logspace(-10, 1, 10, base=2) # 返回13个数字，底是2
print("gamma_rang:", gamma_range)

for gamma_item in gamma_range:
    count=1
    time0 = time()
    print("Start-{0}, gamma={1}".format(count, gamma_item))
    count = count+1
    clf = SVR(kernel = 'rbf', gamma = gamma_item, cache_size=5000)
    clf.fit(x_train_std, y_train)
    
    result = clf.predict(x_test_std)
    # accuracy
    rmse = sqrt(mean_squared_error(y_test, result))
    RMSE_list.append(rmse)
    print("(1) Evaluation - RMSE = ", rmse)
    
    # Spearman
    data = {'result':result, 'y_test':y_test}
    df = pd.DataFrame(data, columns=['result','y_test'])
    spearman = df.corr(method="spearman" )
    print("(2) Evaluation - Spearmman = \n", spearman)
    Spearman_list.append(spearman)
    
    # CCC
    prediction = result
    ground = y_test
    meanfe = getMean(ground)
    meanexp = getMean(prediction)
    meanofdiff = getMeanofDiffs(ground,prediction)
    pvarfe = getPvar(ground, meanfe)
    pvarexp = getPvar(prediction, meanexp)
    ccc = getCCC(pvarfe,pvarexp,meanofdiff,meanfe,meanexp)
    CCC_list.append(ccc)
    print('(3) Evaluation - CCC =  ' + str(ccc))
    print(datetime.datetime.fromtimestamp(time()-time0).strftime("%M:%S:%f"))
    print()
    print()

print("Gamma = ", gamma_range[RMSE_list.index(min(RMSE_list))])
print("RMSE = ", min(RMSE_list))
print("Spearman = ", Spearman_list[RMSE_list.index(min(RMSE_list))])
print("CCC = ", CCC_list[RMSE_list.index(min(RMSE_list))]) 
print(datetime.datetime.fromtimestamp(time()-times_all).strftime("%M:%S:%f"))

gamma_rang: [9.76562500e-04 2.27837703e-03 5.31558594e-03 1.24015707e-02
 2.89335848e-02 6.75037337e-02 1.57490131e-01 3.67433623e-01
 8.57243983e-01 2.00000000e+00]
Start-1, gamma=0.0009765625
(1) Evaluation - RMSE =  0.24946551052440782
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.585983
y_test  0.585983  1.000000
(3) Evaluation - CCC =  0.5897413514995558
00:00:261725


Start-1, gamma=0.0022783770304221013
(1) Evaluation - RMSE =  0.2496876487035404
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.585041
y_test  0.585041  1.000000
(3) Evaluation - CCC =  0.6040868420414233
00:00:246915


Start-1, gamma=0.005315585938181161
(1) Evaluation - RMSE =  0.2504135673577296
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.584531
y_test  0.584531  1.000000
(3) Evaluation - CCC =  0.6112775279367313
00:00:275338


Start-1, gamma=0.012401570718501561
(1) Evaluation - RMSE =  0.2514588498456454
(2) Evalua

# Arousal

In [9]:
df = pd.read_csv('Continuous_fusion_4/fusion_4_arousal.csv')
df

,opensmile_prediction_a,hog_prediction_a,vggish_prediction_a,lstm_a_prediction,a_groundtruth
0,0.084600,0.042477,-0.019268,0.025110,0.0
1,-0.127861,-0.358669,0.235731,-0.388250,-0.5
2,-0.054005,0.615145,0.141468,0.844878,1.0
3,0.257527,0.793447,0.635042,0.863487,1.0
4,-0.004294,0.701020,-0.134697,0.879901,0.5
...,...,...,...,...,...
4380,-0.028904,-0.085786,0.053309,-0.170534,0.0
4381,0.317954,-0.014591,0.012568,-0.009125,0.0
4382,-0.054132,-0.369154,-0.135510,-0.273014,-0.5
4383,0.050987,-0.026982,0.005464,-0.000961,0.0


In [10]:
x = df.iloc[:, :-1]  # 数据特征
y = df.iloc[:,-1]  # 标签
# 将数据划分为训练集和测试集，test_size=.3表示30%的测试集, 随机数种子, 保证可复现性
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=420)

# 修正测试集和训练集的索引
for i in [x_train, x_test, y_train, y_test ]:
    i.index  = range(i.shape[0])

# 标准化
scaler_x = StandardScaler()
# scaler_y = StandardScaler()
x_train_std = scaler_x.fit_transform(x_train)
x_test_std = scaler_x.fit_transform(x_test)
# y_train = scaler_y.fit_transform(np.array(y_train).reshape([-1,1])).reshape(-1)
# y_test = scaler_y.fit_transform(np.array(y_test).reshape([-1,1])).reshape(-1)
print(x_train_std.shape)
print(y_train.shape)

(3069, 4)
(3069,)


In [11]:
times_all = time()
# 调试两个参数 gamma & C  ，默认情况下C为1，通常来说这都是一个合理的参数。
RMSE_list = []
Spearman_list = []
CCC_list = []

gamma_range = np.logspace(-10, 1, 10, base=2) # 返回13个数字，底是2
print("gamma_rang:", gamma_range)

for gamma_item in gamma_range:
    count=1
    time0 = time()
    print("Start-{0}, gamma={1}".format(count, gamma_item))
    count = count+1
    clf = SVR(kernel = 'rbf', gamma = gamma_item, cache_size=5000)
    clf.fit(x_train_std, y_train)
    
    result = clf.predict(x_test_std)
    # accuracy
    rmse = sqrt(mean_squared_error(y_test, result))
    RMSE_list.append(rmse)
    print("(1) Evaluation - RMSE = ", rmse)
    
    # Spearman
    data = {'result':result, 'y_test':y_test}
    df = pd.DataFrame(data, columns=['result','y_test'])
    spearman = df.corr(method="spearman" )
    print("(2) Evaluation - Spearmman = \n", spearman)
    Spearman_list.append(spearman)
    
    # CCC
    prediction = result
    ground = y_test
    meanfe = getMean(ground)
    meanexp = getMean(prediction)
    meanofdiff = getMeanofDiffs(ground,prediction)
    pvarfe = getPvar(ground, meanfe)
    pvarexp = getPvar(prediction, meanexp)
    ccc = getCCC(pvarfe,pvarexp,meanofdiff,meanfe,meanexp)
    CCC_list.append(ccc)
    print('(3) Evaluation - CCC =  ' + str(ccc))
    print(datetime.datetime.fromtimestamp(time()-time0).strftime("%M:%S:%f"))
    print()
    print()

print("Gamma = ", gamma_range[RMSE_list.index(min(RMSE_list))])
print("RMSE = ", min(RMSE_list))
print("Spearman = ", Spearman_list[RMSE_list.index(min(RMSE_list))])
print("CCC = ", CCC_list[RMSE_list.index(min(RMSE_list))]) 
print(datetime.datetime.fromtimestamp(time()-times_all).strftime("%M:%S:%f"))

gamma_rang: [9.76562500e-04 2.27837703e-03 5.31558594e-03 1.24015707e-02
 2.89335848e-02 6.75037337e-02 1.57490131e-01 3.67433623e-01
 8.57243983e-01 2.00000000e+00]
Start-1, gamma=0.0009765625
(1) Evaluation - RMSE =  0.18661911565683995
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.630151
y_test  0.630151  1.000000
(3) Evaluation - CCC =  0.8538523693929926
00:00:213633


Start-1, gamma=0.0022783770304221013
(1) Evaluation - RMSE =  0.1868437138814836
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.630264
y_test  0.630264  1.000000
(3) Evaluation - CCC =  0.8561791754177868
00:00:201054


Start-1, gamma=0.005315585938181161
(1) Evaluation - RMSE =  0.18637555091270125
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.629629
y_test  0.629629  1.000000
(3) Evaluation - CCC =  0.8571619337596973
00:00:227736


Start-1, gamma=0.012401570718501561
(1) Evaluation - RMSE =  0.18556072825975084
(2) Eval

# Dominance

In [12]:
df = pd.read_csv('Continuous_fusion_4/fusion_4_dominance.csv')
df

,opensmile_prediction_d,hog_prediction_d,vggish_prediction_d,d_prediction,d_groundtruth
0,-0.079165,0.047263,0.093845,0.001057,0.0000
1,-0.042976,-0.014794,0.336009,-0.001182,0.0000
2,-0.080992,0.542729,-0.001804,0.528849,0.3333
3,0.322125,0.785058,0.456919,0.737339,0.6667
4,0.039425,0.585995,-0.067457,0.885210,0.3333
...,...,...,...,...,...
4380,0.071286,0.065583,0.019601,-0.000266,0.0000
4381,0.342883,0.013817,0.074017,0.006694,0.0000
4382,0.052191,0.045060,0.203588,-0.001473,0.0000
4383,0.079865,-0.101391,0.299809,-0.001474,0.0000


In [13]:
x = df.iloc[:, :-1]  # 数据特征
y = df.iloc[:,-1]  # 标签
# 将数据划分为训练集和测试集，test_size=.3表示30%的测试集, 随机数种子, 保证可复现性
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=420)

# 修正测试集和训练集的索引
for i in [x_train, x_test, y_train, y_test ]:
    i.index  = range(i.shape[0])

# 标准化
scaler_x = StandardScaler()
# scaler_y = StandardScaler()
x_train_std = scaler_x.fit_transform(x_train)
x_test_std = scaler_x.fit_transform(x_test)
# y_train = scaler_y.fit_transform(np.array(y_train).reshape([-1,1])).reshape(-1)
# y_test = scaler_y.fit_transform(np.array(y_test).reshape([-1,1])).reshape(-1)
print(x_train_std.shape)
print(y_train.shape)

(3069, 4)
(3069,)


In [15]:
times_all = time()
# 调试两个参数 gamma & C  ，默认情况下C为1，通常来说这都是一个合理的参数。
RMSE_list = []
Spearman_list = []
CCC_list = []

gamma_range = np.logspace(-10, 1, 10, base=2) # 返回13个数字，底是2
print("gamma_rang:", gamma_range)

for gamma_item in gamma_range:
    count=1
    time0 = time()
    print("Start-{0}, gamma={1}".format(count, gamma_item))
    count = count+1
    clf = SVR(kernel = 'rbf', gamma = gamma_item, cache_size=5000)
    clf.fit(x_train_std, y_train)
    
    result = clf.predict(x_test_std)
    # accuracy
    rmse = sqrt(mean_squared_error(y_test, result))
    RMSE_list.append(rmse)
    print("(1) Evaluation - RMSE = ", rmse)
    
    # Spearman
    data = {'result':result, 'y_test':y_test}
    df = pd.DataFrame(data, columns=['result','y_test'])
    spearman = df.corr(method="spearman" )
    print("(2) Evaluation - Spearmman = \n", spearman)
    Spearman_list.append(spearman)
    
    # CCC
    prediction = result
    ground = y_test
    meanfe = getMean(ground)
    meanexp = getMean(prediction)
    meanofdiff = getMeanofDiffs(ground,prediction)
    pvarfe = getPvar(ground, meanfe)
    pvarexp = getPvar(prediction, meanexp)
    ccc = getCCC(pvarfe,pvarexp,meanofdiff,meanfe,meanexp)
    CCC_list.append(ccc)
    print('(3) Evaluation - CCC =  ' + str(ccc))
    print(datetime.datetime.fromtimestamp(time()-time0).strftime("%M:%S:%f"))
    print()
    print()

print("Gamma = ", gamma_range[RMSE_list.index(min(RMSE_list))])
print("RMSE = ", min(RMSE_list))
print("Spearman = ", Spearman_list[RMSE_list.index(min(RMSE_list))])
print("CCC = ", CCC_list[RMSE_list.index(min(RMSE_list))]) 
print(datetime.datetime.fromtimestamp(time()-times_all).strftime("%M:%S:%f"))

gamma_rang: [9.76562500e-04 2.27837703e-03 5.31558594e-03 1.24015707e-02
 2.89335848e-02 6.75037337e-02 1.57490131e-01 3.67433623e-01
 8.57243983e-01 2.00000000e+00]
Start-1, gamma=0.0009765625
(1) Evaluation - RMSE =  0.13120818479690408
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.450445
y_test  0.450445  1.000000
(3) Evaluation - CCC =  0.8544196583064825
00:00:154012


Start-1, gamma=0.0022783770304221013
(1) Evaluation - RMSE =  0.13127851752264175
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.450594
y_test  0.450594  1.000000
(3) Evaluation - CCC =  0.8565336885763963
00:00:139807


Start-1, gamma=0.005315585938181161
(1) Evaluation - RMSE =  0.12997833747854118
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.450003
y_test  0.450003  1.000000
(3) Evaluation - CCC =  0.8598166182719806
00:00:141188


Start-1, gamma=0.012401570718501561
(1) Evaluation - RMSE =  0.12705012629410994
(2) Eva